In [28]:
import sys
import numpy as np
import os
import h5py
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [19]:
mainpath = '../../goesdata/sample'

outpath = '../../goesdata/output'

if not os.path.exists(outpath):
    os.makedirs(outpath)
    

In [17]:
def getUniTime(filename):
    splits = filename.split('_')
    return splits[-3][:-3]

file_dict = {}
for root, dirs, files in os.walk(mainpath):
    for file in files:
        print(file)
        uniTime = getUniTime(file)
        if uniTime not in file_dict:
            file_dict[uniTime] = []
        file_dict[uniTime].append(os.path.join(root, file))
                
print(file_dict)


OR_ABI-L1b-RadF-M6C07_G16_s20192210000
OR_ABI-L1b-RadF-M6C07_G17_s20192210000
{'ABI-L1b-RadF-M6': ['../../goesdata/sample/OR_ABI-L1b-RadF-M6C07_G16_s20192210000', '../../goesdata/sample/OR_ABI-L1b-RadF-M6C07_G17_s20192210000']}


In [18]:
#Takes the urls for GOES16 and GOES17 from the same time
def genLabel(goes16, goes17):
    goes16 = np.load(goes16)
    goes17 = np.load(goes17)

    n1 = len(goes16)*len(goes16[0])
    n2 = len(goes17)*len(goes17[0])

    std1 = np.std(goes16)
    std2 = np.std(goes17)

    diff = goes16 - goes17
    diff_std = np.std(diff)

    #joint_std = np.sqrt(std1**2/n1 + std2**2/n2)
    delta = abs(goes16 - goes17)
    label = delta//diff_std
    label = label.astype(int)
    
    return label


In [20]:
#Outputs the data of G17 and the generated label in HDF5
def writeH5(data, label, uniTime):
    coords = np.array(data).astype(np.float)
    shape = coords.shape
    hdf5_path = outpath + uniTime + '.h5'
    with h5py.File(hdf5_path, mode='w') as f:
        d = f.create_dataset('/data', data = coords)
        l = f.create_dataset('/label', data = label)
        

In [25]:
labels = None
for key in file_dict:
    pair = file_dict[key]
    if len(pair) == 2:
        print(pair)
        labels = genLabel(pair[0], pair[1])
        writeH5(np.load(pair[1]), labels, key)

['../../goesdata/sample/OR_ABI-L1b-RadF-M6C07_G16_s20192210000', '../../goesdata/sample/OR_ABI-L1b-RadF-M6C07_G17_s20192210000']


In [26]:
out = outpath + 'label1.txt'
np.savetxt(out, labels, fmt='%10.f')

In [27]:
labels

array([[1, 2, 1, ..., 0, 1, 0],
       [2, 2, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 2, 2, 2],
       [1, 0, 0, ..., 1, 1, 3],
       [1, 0, 0, ..., 0, 1, 2]])

In [30]:
df = pd.DataFrame(labels)

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,1,2,1,1,1,0,1,0,1,2,...,0,0,1,0,0,0,0,0,1,0
1,2,2,1,1,0,0,1,1,2,1,...,0,0,0,0,0,0,1,0,1,1
2,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,1,1
3,1,2,2,1,0,1,1,1,0,1,...,0,1,0,0,0,2,1,0,1,0
4,1,2,2,1,1,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
5,2,1,1,1,1,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
7,1,1,1,1,1,1,1,1,1,1,...,0,0,1,1,1,0,0,1,1,0
8,2,0,1,1,1,1,1,1,1,1,...,0,0,0,2,1,1,0,0,0,0
9,2,0,0,1,1,1,0,1,1,1,...,0,0,0,0,1,1,0,0,1,0


In [33]:
myNumpy = np.load('../../goesdata/DQF/221/OR_ABI-L1b-RadF-M6C07_G16_s20192210000')
dfNOAA = pd.DataFrame(myNumpy)
dfNOAA

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:

#URL = 'https://github.com/adomakor412/GOES-Quality-Flag-Classification/blob/master/G16_G17_url_list/G16_data_08_09-19.txt'
goes16_urls = []
goes17_urls = []
for s in range(16,18):
    SS = str(s).zfill(2)
    for d in range(221, 238):#the website for particular day
        DDD = str(d)
        rootURL = f"https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/{DDD}/"
        scraped_urls = scrapeURL(rootURL)
        if SS == '16':
            goes16_urls += scraped_urls
        else:
            goes17_urls += scraped_urls
print(len(goes16_urls))
print(len(goes17_urls))

'from bs4 import BeautifulSoup\nimport requests\n\n#URL = \'https://github.com/adomakor412/GOES-Quality-Flag-Classification/blob/master/G16_G17_url_list/G16_data_08_09-19.txt\'\ngoes16_urls = []\ngoes17_urls = []\nfor s in range(16,18):\n    SS = str(s).zfill(2)\n    for d in range(221, 238):#the website for particular day\n        DDD = str(d)\n        rootURL = f"https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/{DDD}/"\n        scraped_urls = scrapeURL(rootURL)\n        if SS == \'16\':\n            goes16_urls += scraped_urls\n        else:\n            goes17_urls += scraped_urls\nprint(len(goes16_urls))\nprint(len(goes17_urls))'

In [71]:
'''rootURL = "https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/"

def listurls(prefix,html):
    soup = BeautifulSoup(html.text)
    urllist = [elt['href'] for elt in soup.find_all(href=re.compile(prefix))]
    return urllist'''

In [72]:
'''
for s in range(16,18):
    SS = str(s).zfill(2)
    for i in range(221,238):#the website for particular day   
        if i < 244:
            mm = '08'
            DD = str(31-(243-i)).zfill(2)
        else:
            mm ='09'
            DD = str(i-243).zfill(2)
        dd = str(i).zfill(2)
        filename = f"G{SS}_data_{mm}_{dd}-19.txt"
        filename.format(**{'SS':SS,'mm':mm,'dd':dd})
        with open(filename,'w') as f:
            for t in range(2):#this is for the hour
                hr = str(t).zfill(2)
                template = f"https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/{DDD}/OR_ABI-L1b-RadF-M6C07_G{SS}_s2019{DDD}{hh}{mm}"

                print(template)
                req = requests.get(template)
                print(req)
                for band in range(8):
                    prefix = "OR_ABI-L1b-RadF-M6C" + str(band).zfill(2)
                    bandURLList = listurls(prefix,req)
                    print(bandURLList)
                    print(*bandURLList,sep='\n', file=f)'''

NameError: name 'hh' is not defined

In [70]:
'''rootURL = "https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/"
page = requests.get(rootURL).text
soup = BeautifulSoup(page, 'html.parser')

url_table = soup.findAll('a')
for link in url_table:  
    if link.has_attr('href'):
            print(link['href'])
#print(url_table[0:]['href'])'''

#content-body
/
/explore
/explore/groups
/explore/snippets
/help
/search?project_id=18540418
/help
https://about.gitlab.com/releases/gitlab-com/
/help
https://about.gitlab.com/getting-help/
https://about.gitlab.com/submit-feedback
https://about.gitlab.com/contributing
https://next.gitlab.com/
/users/sign_in?redirect_to_referer=yes
/adomakor412/goesdata
/adomakor412/goesdata
/adomakor412/goesdata
/adomakor412/goesdata
/adomakor412/goesdata/activity
/adomakor412/goesdata/-/releases
/adomakor412/goesdata/-/tree/master
/adomakor412/goesdata/-/tree/master
/adomakor412/goesdata/-/tree/master
/adomakor412/goesdata/-/commits/master
/adomakor412/goesdata/-/branches
/adomakor412/goesdata/-/tags
/adomakor412/goesdata/-/graphs/master
/adomakor412/goesdata/-/network/master
/adomakor412/goesdata/-/compare?from=master&to=master
/adomakor412/goesdata/path_locks
/adomakor412/goesdata/-/issues
/adomakor412/goesdata/-/issues
/adomakor412/goesdata/-/issues
/adomakor412/goesdata/-/boards
/adomakor412/goesd

In [111]:
dataSource = np.DataSource(None)
response = np.DataSource('https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/OR_ABI-L1b-RadF-M6C07_G16_s20192210000548_e20192210010268_c20192210010328.npy')

In [113]:
response.open('https://gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/OR_ABI-L1b-RadF-M6C07_G16_s20192210000548_e20192210010268_c20192210010328.npy')

<_io.TextIOWrapper name='/Users/aleex/DSE/Machine Learning/Project/https:/gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/OR_ABI-L1b-RadF-M6C07_G16_s20192210000548_e20192210010268_c20192210010328.npy/gitlab.com/adomakor412/goesdata/-/blob/master/fall_npy/221/OR_ABI-L1b-RadF-M6C07_G16_s20192210000548_e20192210010268_c20192210010328.npy' mode='r' encoding='UTF-8'>

In [114]:
print(response)